In [15]:
import pandas as pd
from scipy.stats import skew, kurtosis
import csv

In [34]:
def imf_mean(imf): 
    mean = imf.mean()
    return mean[1]

In [35]:
def imf_std(imf):
    std = imf.std()
    return std[1]

In [36]:
def imf_kurtosis(imf):
    k = kurtosis(imf.iloc[:, [1]])
    return (k[0])

In [37]:
def imf_skew(imf):
    s = skew(imf.iloc[:, [1]])
    return (s[0])

In [38]:
def add_row(imf, start, end, duration, seizure):
    row_to_add = []
    i = imf[start:end]
    
    i_mean = imf_mean(i)
    i_std = imf_std(i) 
    i_kurtosis = imf_kurtosis(i) 
    i_skew = imf_skew(i) 
    
    row_to_add.append(i_mean)
    row_to_add.append(i_std)
    row_to_add.append(i_kurtosis)
    row_to_add.append(i_skew)
    
    # Label: 1 = seizure, 0 = non-seizure. Change before running.
    row_to_add.append(seizure)

    return row_to_add

In [45]:
def write_to_file(file, start, end, temp, imf, seizure):
    duration = 10*256
    res = pd.DataFrame()

    # first iteration run in 'w' mode, all subsequent iteration run in 'a' mode
    with open(file, 'a') as file:
        writer = csv.writer(file)
        while temp < end:
            row = add_row(imf, temp, temp + duration, duration, seizure)
            res = res.append(pd.Series(row), ignore_index=True)
            writer.writerow(row)
            temp += duration

    print (res)

In [40]:
#patient 1
chb01 = ['chb01_03', 'chb01_04', 'chb01_15', 'chb01_16', 'chb01_18', 'chb01_21','chb01_26']
#patient 8
chb08 = ['chb08_02', 'chb08_05', 'chb08_11', 'chb08_13', 'chb08_21']
#patient 11 //done
chb11 = ['chb11_82', 'chb11_92', 'chb11_99']
#patient 14
chb14 = ['chb14_03', 'chb14_04', 'chb14_06', 'chb14_11', 'chb14_17', 'chb14_18', 'chb14_27']
#patient 14
chb20 = ['chb20_12', 'chb20_13', 'chb20_14', 'chb20_15', 'chb20_16', 'chb20_68']   

In [59]:
k = pd.read_csv('main_trimmed.csv')
chb11 = k.iloc[12:15, :]
chb11

,S. No.,Patient,File,Start,End,Duration
12,56,chb11,chb11_82,300,320,20
13,57,chb11,chb11_92,2697,2727,30
14,58,chb11,chb11_99,1454,2204,750


In [60]:
for index, row in chb11.iterrows():
        print(index, row.Patient, row.File)
        imf = pd.read_csv('IMF/chb11/{file}_imf.csv'.format(file = row.File))
        start = temp = row.Start*256
        end = row.End*256
        write_to_file('{}_imf_fe.csv'.format(row.File), start, end, temp, imf, 1)

12 chb11 chb11_82
              0         1          2         3    4
0 -3.022630e-09  0.000001  13.049274  0.146507  1.0
1 -4.319687e-09  0.000002  58.569121 -0.048327  1.0
13 chb11 chb11_92
              0             1          2         3    4
0  1.670261e-09  8.554492e-07   2.477585  0.052177  1.0
1  7.193723e-09  1.254036e-06   6.705502  0.081010  1.0
2 -1.278357e-09  1.870227e-06  41.778794 -0.306958  1.0
14 chb11 chb11_99
               0             1          2         3    4
0  -6.702095e-10  9.882789e-07  21.861927  0.453900  1.0
1   1.866179e-08  1.794939e-06  24.057997  0.230653  1.0
2   2.063364e-09  2.208709e-06   8.480762  0.082124  1.0
3  -1.623917e-08  3.064081e-06  21.109577  0.026857  1.0
4  -9.310539e-09  4.381313e-06  45.643097  0.536828  1.0
..           ...           ...        ...       ...  ...
70 -1.273930e-09  4.625543e-07   6.905415  0.079185  1.0
71  2.998184e-09  6.418206e-07  90.960276 -0.205898  1.0
72  7.517083e-09  4.516863e-07   0.924814  0.028104  

In [65]:
f82 = pd.read_csv('chb11_82_imf_fe.csv',header=None)
f92 = pd.read_csv('chb11_92_imf_fe.csv',header=None)
f99 = pd.read_csv('chb11_99_imf_fe.csv',header=None)

In [71]:
f82

(175, 5)

In [72]:
c = [f82, f92, f99]
j = pd.concat(c, sort=False)
j.columns = ['IMF_Mean', 'IMF_Std','IMF_Kurtosis','IMF_Skew', 'Label']
j.to_csv('chb11_imf_features.csv')